<a href="https://colab.research.google.com/github/nbht98/attack_cat_or_dog_classification/blob/master/TEST_CIFAR_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip3 install mia

  Created wheel for mia: filename=mia-0.1.2-cp36-none-any.whl size=11079 sha256=072375c16f60d8b8af98633d20a9dfe9c4c4870a2377246d84cfbf8c4754df6a
  Stored in directory: /root/.cache/pip/wheels/e8/83/e4/baae7782aa0d2e45af485d25a7994bab3f76428e483252ce82
Successfully built mia


In [0]:
# Import library 
import numpy as np

from absl import app
from absl import flags

import tensorflow as tf
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

from mia.estimators import ShadowModelBundle, AttackModelBundle, prepare_attack_data


In [0]:
NUM_CLASSES = 10
WIDTH = 32
HEIGHT = 32
CHANNELS = 3

TARGET_EPOCHS = 100 # Number of epochs to train target and shadow models
ATTACK_EPOCHS = 100 # Number of epochs to train attack models.
NUM_SHADOWS = 3 # Number of epochs to train attack models.

In [0]:
def get_data():
    """Prepare CIFAR10 data."""
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
    y_train = tf.keras.utils.to_categorical(y_train)
    y_test = tf.keras.utils.to_categorical(y_test)
    X_train = X_train.astype("float32")
    X_test = X_test.astype("float32")
    y_train = y_train.astype("float32")
    y_test = y_test.astype("float32")
    X_train /= 255
    X_test /= 255
    return (X_train, y_train), (X_test, y_test)


In [0]:
def target_model_fn():
  """The architecture of the target (victim) model.
  The attack is white-box, hence the attacker is assumed to know this architecture too."""

  model = tf.keras.models.Sequential()

  model.add(
      layers.Conv2D(
          32,
          (3, 3),
          activation="relu",
          padding="same",
          input_shape=(WIDTH, HEIGHT, CHANNELS),
      )
  )
  model.add(layers.Conv2D(32, (3, 3), activation="relu"))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Dropout(0.25))

  model.add(layers.Conv2D(64, (3, 3), activation="relu", padding="same"))
  model.add(layers.Conv2D(64, (3, 3), activation="relu"))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Dropout(0.25))

  model.add(layers.Flatten())

  model.add(layers.Dense(512, activation="relu"))
  model.add(layers.Dropout(0.5))

  model.add(layers.Dense(NUM_CLASSES, activation="softmax"))
  model.compile("adam", loss="categorical_crossentropy", metrics=["accuracy"])

  return model


def attack_model_fn():
  """Attack model that takes target model predictions and predicts membership.
  Following the original paper, this attack model is specific to the class of the input.
  AttachModelBundle creates multiple instances of this model for each class.
  """
  model = tf.keras.models.Sequential()

  model.add(layers.Dense(128, activation="relu", input_shape=(NUM_CLASSES,)))

  model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
  model.add(layers.Dense(64, activation="relu"))
  model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
  model.add(layers.Dense(64, activation="relu"))

  model.add(layers.Dense(1, activation="sigmoid"))
  model.compile("adam", loss="binary_crossentropy", metrics=["accuracy"])
  return model


In [9]:
(X_train, y_train), (X_test, y_test) = get_data()

170500096/170498071 [==============================] - 6s 0us/step


In [17]:
# Train the target model.
print("Training the target model...")
target_model = target_model_fn()
target_model.load_weights("model1500.h5")

Training the target model...


In [0]:
target_model.fit(
    X_train, y_train, epochs=300, validation_split=0.1, verbose=True
)

In [0]:
target_model.save('model1500.h5')

In [0]:
# import matplotlib.pyplot as plt
# import numpy as np

# plt.imshow(np.squeeze(X_train[0]))


In [0]:
def rand_array(x, k, in_channels, img_size):
    total_num = in_channels * img_size * img_size
    x = x.reshape(1, -1)
    idx_to_modify = np.random.randint(low=0, high=total_num-1, size=(k,))
    gen = np.random.rand(k)
    x[0, idx_to_modify] = gen
    x = x.reshape(img_size, img_size, in_channels)
    return x

In [0]:
def rand_array_delta(x, k, in_channels, img_size, delta):
    total_num = in_channels * img_size * img_size
    x = x.reshape(1, -1)
    idx_to_modify = np.random.randint(low=0, high=total_num-1, size=(k,))
    x[0, idx_to_modify] += delta
    for i in range(32*32*3):
      if x[0, [i]] > 1:
        x[0, [i]] = x[0, [i]] - 1
    x = x.reshape(img_size, img_size, in_channels)
    return x

In [0]:
def data_synthesize(target_model, fixed_cls, k_max,
                    in_channels, img_size):
    x = np.random.rand(32,32,3)
    y_current = np.zeros((1,10))
    y_c_current = 0         # target models probability of fixed class
    j = 0                   # consecutive rejections counter
    k = k_max               # search radius
    max_iter = 50           # max iter number
    conf_min = 0.9          # min probability cutoff to consider a record member of the class
    rej_max = 5             # max number of consecutive rejections
    k_min = 1               # min radius of feature perturbation
    x_new = x
    for i in range(max_iter):
        y = target_model.predict(x.reshape(1,32,32,3))  # query target model
        y_c = y.flat[fixed_cls]
        e = np.max(y)
        # Phase 1: Search
        if y_c > y_c_current:
            # Phase 2: Sample
            e = np.max(y - y_current)
            # print(i, y_c, e)
            if (y_c > conf_min) and (fixed_cls == np.argmax(y)):
                return (x, y)
            x_new = x
            y_c_current = y_c  # renew variables
            y_current = y
            j = 0
        else:
            j += 1
            if j > rej_max:  # many consecutive rejects
                k = max(k_min, int(np.ceil(k / 2)))
                j = 0
        # x = rand_array(x_new, k, in_channels, img_size)
        x = rand_array_delta(x_new, k, in_channels, img_size, e)

    return False

In [0]:
while True:
  data = data_synthesize(target_model, fixed_cls=8, k_max=3*32*32-1,
                    in_channels=3, img_size=32)
  if isinstance(data, tuple):
    break
 
print("########## Example ##########")
print("x = ", data[0])
print("y_c = ", data[1])



In [0]:
from numpy import savetxt
def synthesize_batch(target_model, lst_class, n_records):
    """
    Synthesize a batch of records
    """
    x_synth = np.zeros((n_records,32,32,3))
    y_synth = np.zeros((n_records, 10))
    count = 0
    for i in range(n_records//10):
      for j in lst_class:
        print(j, end=' ')
        while True:
            data = data_synthesize(target_model, fixed_cls=j, k_max=3*32*32-1,
                          in_channels=3, img_size=32)
            if isinstance(data, tuple):
                break
      
        x_synth[count] = data[0]
        y_synth[count] = data[1]
        count += 1
      print()
      if (i % 2 == 1 and i != 0):
        np.savez('data' + str(i) +'_90.npz', x=x_synth, y=y_synth)
        print('data'+ str(i) +' has been saved')
    return x_synth, y_synth

In [18]:
attack_data = synthesize_batch(target_model, [0,1,2,3,4,5,6,7,8,9], 100)

0 1 2 3 4 5 6 7 8 9 
0 1 2 3 4 5 6 7 8 9 
data1 has been saved
0 1 2 3 4 5 6 7 8 9 
0 1 2 3 4 5 6 7 8 9 
data3 has been saved
0 1 2 3 4 5 6 7 8 9 
0 1 2 3 4 5 6 7 8 9 
data5 has been saved
0 1 2 3 4 5 6 7 8 9 
0 1 2 3 4 5 6 7 8 9 
data7 has been saved
0 1 2 3 4 5 6 7 8 9 
0 1 2 3 4 5 6 7 8 9 
data9 has been saved


In [0]:
np.savez('data100_90.npz', x=attack_data[0], y=attack_data[1])

In [0]:
from numpy import load 
a_d = np.load('data100_90.npz')

In [21]:
# We assume that attacker's data were not seen in target's training.
attacker_X_train, attacker_X_test, attacker_y_train, attacker_y_test = train_test_split(
    attack_data[0], attack_data[1], test_size=0.1
)
print(attacker_X_train.shape, attacker_X_test.shape)

(90, 32, 32, 3) (10, 32, 32, 3)


In [95]:
attacker_X_train, attacker_X_test, attacker_y_train, attacker_y_test = train_test_split(
    a_d['x'], a_d['y'], test_size=0.1
)
print(attacker_X_train.shape, attacker_X_test.shape)

(90, 32, 32, 3) (10, 32, 32, 3)


In [0]:
SHADOW_DATASET_SIZE = 45

# Train the shadow models.
smb = ShadowModelBundle(
    target_model_fn,
    shadow_dataset_size=SHADOW_DATASET_SIZE,
    num_models=NUM_SHADOWS,
)

In [37]:
print("Training the shadow models...")
X_shadow, y_shadow = smb.fit_transform(
    attacker_X_train,
    attacker_y_train,
    fit_kwargs=dict(
        epochs=100,
        verbose=True,
        validation_data=(attacker_X_test, attacker_y_test),
    ),
)

Training the shadow models...
Epoch 1/100
2/2 [==============================] - 0s 91ms/step - loss: 2.4180 - accuracy: 0.0222 - val_loss: 2.4085 - val_accuracy: 0.0000e+00
Epoch 2/100
2/2 [==============================] - 0s 23ms/step - loss: 2.2725 - accuracy: 0.1556 - val_loss: 2.4097 - val_accuracy: 0.0000e+00
Epoch 3/100
2/2 [==============================] - 0s 24ms/step - loss: 2.2207 - accuracy: 0.1778 - val_loss: 2.4320 - val_accuracy: 0.0000e+00
Epoch 4/100
2/2 [==============================] - 0s 24ms/step - loss: 2.2814 - accuracy: 0.1333 - val_loss: 2.4516 - val_accuracy: 0.2000
Epoch 5/100
2/2 [==============================] - 0s 25ms/step - loss: 2.2405 - accuracy: 0.1556 - val_loss: 2.4450 - val_accuracy: 0.2000
Epoch 6/100
2/2 [==============================] - 0s 23ms/step - loss: 2.2770 - accuracy: 0.1111 - val_loss: 2.4110 - val_accuracy: 0.2000
Epoch 7/100
2/2 [==============================] - 0s 22ms/step - loss: 2.2250 - accuracy: 0.1778 - val_loss: 2.3969 -

In [38]:
# ShadowModelBundle returns data in the format suitable for the AttackModelBundle.
amb = AttackModelBundle(attack_model_fn, num_classes=NUM_CLASSES)

# Fit the attack models.
print("Training the attack models...")
amb.fit(
    X_shadow, y_shadow, fit_kwargs=dict(epochs=50, verbose=True)
)

Training the attack models...
Epoch 1/50
1/1 [==============================] - 0s 3ms/step - loss: 0.6799 - accuracy: 0.8333
Epoch 2/50
1/1 [==============================] - 0s 2ms/step - loss: 0.6702 - accuracy: 0.6667
Epoch 3/50
1/1 [==============================] - 0s 2ms/step - loss: 0.6640 - accuracy: 0.7667
Epoch 4/50
1/1 [==============================] - 0s 1ms/step - loss: 0.6598 - accuracy: 0.8333
Epoch 5/50
1/1 [==============================] - 0s 1ms/step - loss: 0.6509 - accuracy: 0.8667
Epoch 6/50
1/1 [==============================] - 0s 1ms/step - loss: 0.6421 - accuracy: 0.8667
Epoch 7/50
1/1 [==============================] - 0s 1ms/step - loss: 0.6314 - accuracy: 0.8000
Epoch 8/50
1/1 [==============================] - 0s 2ms/step - loss: 0.6081 - accuracy: 0.9000
Epoch 9/50
1/1 [==============================] - 0s 2ms/step - loss: 0.6055 - accuracy: 0.8667
Epoch 10/50
1/1 [==============================] - 0s 3ms/step - loss: 0.6046 - accuracy: 0.8333
Epoch 11/

In [39]:
# Test the success of the attack.
ATTACK_TEST_DATASET_SIZE = 4000
# Prepare examples that were in the training, and out of the training.
data_in = X_train[:ATTACK_TEST_DATASET_SIZE], y_train[:ATTACK_TEST_DATASET_SIZE]
data_out = X_test[:ATTACK_TEST_DATASET_SIZE], y_test[:ATTACK_TEST_DATASET_SIZE]
# data_out = np.concatenate((X_test, a_X_train), axis=0), np.concatenate((y_test, a_y_train), axis=0)


# Compile them into the expected format for the AttackModelBundle.
attack_test_data, real_membership_labels = prepare_attack_data(
    target_model, data_in, data_out
)

# Compute the attack accuracy.
attack_guesses = amb.predict(attack_test_data)
attack_accuracy = np.mean(attack_guesses == real_membership_labels)
print(attack_accuracy)


0.615875
